In [ ]:
from tufast_matching_tool.matching import read_sheet
from tufast_matching_tool.maxflow import *

In [ ]:
df,shift_type_dict = read_sheet("../Shiftsplan_lux025.xlsx","ShiftList_KW16")

In [ ]:
shift_degree = [df[s].sum() for s in shift_type_dict]
len([x for x in shift_degree if x < 2])
for i,key in enumerate(shift_type_dict):
    if shift_degree[i] < 2:
        shift_type_dict[key] = []

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)
base_mat = shiftplan_to_adj_mat(df)
shift_type_dict_copy = shift_type_dict.copy()
for i in range(20):
    sl_mat = make_sl_adj_mat(df,base_mat,shift_type_dict_copy)
    flow_sl = get_flow_from_adj_mat(sl_mat)
    w_mat = make_worker_adj_mat(df,base_mat,flow_sl,2)
    flow = get_flow_from_adj_mat(w_mat)

    shift_list = get_shift_list(df,flow_sl,flow)

    print(flow_sl.flow_value+flow.flow_value)
    
    for k in shift_list:
        if (shift_list[k]["lead"] is not None) and (shift_list[k] is None or len(shift_list[k]["worker"]) == 0):
            shift_type_dict_copy[k] = []
            shift_list = None
            break
    
    if shift_list is not None:
        break

w_mat_2 = make_worker_adj_mat(df,w_mat,flow,2)
flow_2 = get_flow_from_adj_mat(w_mat_2)
shift_list = get_shift_list(df,flow_sl,flow,flow_2)
print(flow_sl.flow_value+flow.flow_value+flow_2.flow_value)

In [ ]:
n_shift_res = {}
for n in list(df["name"]):
    n_shift_res[n] = int(df[df["name"]==n]["n_shifts"])
for k in shift_list:
    print(k)
    if shift_list[k]["lead"] is None:
        print("None")
    else:
        print(shift_list[k]["lead"])
        n_shift_res[shift_list[k]["lead"]] -= 1
        for m in shift_list[k]["worker"]:
            n_shift_res[m] -= 1
            print(m)
    print()

In [ ]:
for n in n_shift_res:
    if n_shift_res[n] > 0:
        print(n,n_shift_res[n])

In [ ]:
draw_matching(df,base_mat)

In [ ]:
sl_mat = make_sl_adj_mat(df,base_mat,shift_type_dict)
draw_matching(df,sl_mat)

In [ ]:
draw_matching(df,flow_sl.flow)

In [ ]:
draw_matching(df,flow.flow)